<a href="https://colab.research.google.com/github/Khamdanfakhryza/optimasi-jaringan-penyulang/blob/main/metode_optimasi_untuk_jaringan_penyulang_20_kV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#----------------------------Import Modul yang Diperlukan
import pandas as pd
import numpy as np
from google.colab import files
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
import csv
from datetime import datetime, timedelta
from google.colab import files

# Create an empty list to store data
data = []

# Determine the number of nodes and time period
jumlah_node = 50  # increase the number of nodes to get more comprehensive data
tanggal_mulai = datetime(2023, 1, 1)
hari_tercatat = 365  # data is taken for one year to cover a longer period

# Create a loop to fill in the data
for day in range(hari_tercatat):
    for i in range(1, jumlah_node + 1):
        node = i
        r = 0.05 * i  # optimize R value to reduce losses
        x = 0.05 * i + 0.05  # optimize X value to reduce losses
        p = 0.5 * i + 0.5  # optimize P value to increase revenue
        q = 0.1 * i + 0.1  # optimize Q value to increase revenue
        tanggal = tanggal_mulai + timedelta(days=day)
        data.append([tanggal.strftime('%Y-%m-%d'), node, r, x, p, q])

# Write data to CSV file
file_path = '/content/data_jaringan_penyulang_20kV.csv'
with open(file_path, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["Tanggal", "Node", "R (ohm)", "X (ohm)", "P (MW)", "Q (MVAR)"])
    writer.writerows(data)

# Provide a link to download the file
files.download(file_path)

# Calculate revenue and losses
revenue = 0
losses = 0
for row in data:
    p = row[4]
    q = row[5]
    revenue += p * 1500  # assume 1500 Rupiah per MWh
    losses += (p ** 2 + q ** 2) * 0.05  # assume 5% loss rate

print(f"Total revenue: Rp {revenue:.2f}")
print(f"Total losses: Rp {losses:.2f}")
print(f"Net profit: Rp {revenue - losses:.2f}")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Total revenue: Rp 362718750.00
Total losses: Rp 216016.13
Net profit: Rp 362502733.88


In [ ]:
#----------------------------------Install PyDrive
!pip install -U -q PyDrive


In [ ]:
#----------------------------Autentikasi dan Unduh File dari Google Drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# ID file dari Google Drive
file_id = '1tyMA6OWYI0SP-YxTfXbh8go18sSPdj3J'

# Dapatkan file
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('data.csv')  # nama file yang diunduh disimpan sebagai 'data.csv'

# Baca data dari file CSV
data = pd.read_csv('data.csv')

# Tampilkan data untuk memastikan file terbaca dengan baik
print(data.head())


In [ ]:
#-------------------------------------------------------------Persiapan Data

# Data impedansi saluran (misalnya dalam bentuk matriks admitansi Ybus)
Ybus = np.array([[complex(10, -5), complex(-5, 2)],
                 [complex(-5, 2), complex(8, -3)]])

# Data beban pada setiap node (misalnya dalam bentuk daya nyata dan daya reaktif)
P_load = np.array([0, 200])  # misalnya beban di node 1 adalah 200 kW
Q_load = np.array([0, 100])  # misalnya beban di node 1 adalah 100 kVAR

# Tegangan awal pada setiap node (V) - inisialisasi lebih realistis
V = np.array([complex(1, 0), complex(0.95, 0.01)])  # Asumsi tegangan awal


In [ ]:
#-----------Langkah 2: Implementasi Metode Gauss-Seidel
def gauss_seidel(Ybus, P_load, Q_load, V, tol=1e-6, max_iter=1000):
    """
    Fungsi untuk metode Gauss-Seidel.
    """
    num_bus = len(V)
    for iteration in range(max_iter):
        V_new = np.copy(V)
        for i in range(1, num_bus):
            sum_YV = sum(Ybus[i, j] * V[j] for j in range(num_bus) if j != i)
            V_new[i] = (P_load[i] - 1j * Q_load[i]) / np.conj(V[i]) - sum_YV
            V_new[i] /= Ybus[i, i]
        # Check for convergence
        if np.allclose(V, V_new, atol=tol):
            print(f"Converged in {iteration+1} iterations.")
            break
        V = V_new
    else:
        print("Did not converge within the maximum number of iterations.")
    return V

# Jalankan metode Gauss-Seidel
V_final = gauss_seidel(Ybus, P_load, Q_load, V)
print("Tegangan akhir pada setiap node:")
print(V_final)


Converged in 177 iterations.
Tegangan akhir pada setiap node:
[1.       +0.j        2.9092329-4.4110229j]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#------------------------------Langkah 3: Perhitungan Losses
def calculate_losses(Ybus, V):
    """
    Fungsi untuk menghitung losses dalam jaringan.
    """
    losses = 0
    for i in range(len(Ybus)):
        for j in range(len(Ybus)):
            if i != j:
                losses += np.abs(V[i] - V[j])**2 * np.real(Ybus[i, j])
    return losses

# Hitung losses
total_losses = calculate_losses(Ybus, V_final)
print(f"Total losses dalam jaringan: {total_losses} kW")


Total losses dalam jaringan: -231.02293349740467 kW


In [ ]:
#--------------------------------Langkah 4: Evaluasi Hasil
def calculate_revenue_loss_reduction(initial_losses, optimized_losses, cost_per_kW=1000):
    """
    Fungsi untuk menghitung peningkatan pendapatan berdasarkan pengurangan losses.
    """
    reduction = initial_losses - optimized_losses
    revenue_increase = reduction * cost_per_kW
    return revenue_increase

# Asumsi losses awal (misalnya dari data sebelumnya)
initial_losses = 500  # kW, ini adalah contoh, sesuaikan dengan data nyata

# Hitung peningkatan pendapatan
revenue_increase = calculate_revenue_loss_reduction(initial_losses, total_losses)
print(f"Peningkatan pendapatan PLN UP3 Semarang: Rp {revenue_increase}")

# Simpan hasil ke file teks
with open('result.txt', 'w') as f:
    f.write("Tegangan akhir pada setiap node:\n")
    for v in V_final:
        f.write(f"{v}\n")

# Simpan losses dan peningkatan pendapatan ke file CSV
results_df = pd.DataFrame({
    'initial_losses': [initial_losses],
    'optimized_losses': [total_losses],
    'revenue_increase': [revenue_increase]
})
results_df.to_csv('results.csv', index=False)

Peningkatan pendapatan PLN UP3 Semarang: Rp 731022.9334974047


In [ ]:
#--------------------------------Implementasi Metode Jacobi
def jacobi(Ybus, P_load, Q_load, V, tol=1e-6, max_iter=1000):
    """
    Fungsi untuk metode Jacobi.
    """
    num_bus = len(V)
    for iteration in range(max_iter):
        V_new = np.copy(V)
        for i in range(1, num_bus):
            sum_YV = sum(Ybus[i, j] * V[j] for j in range(num_bus) if j != i)
            V_new[i] = ((P_load[i] - 1j * Q_load[i]) / np.conj(V[i]) - sum_YV) / Ybus[i, i]
        # Check for convergence
        if np.allclose(V, V_new, atol=tol):
            print(f"Jacobi converged in {iteration+1} iterations.")
            break
        V = V_new
    else:
        print("Jacobi did not converge within the maximum number of iterations.")
    return V


In [ ]:
#----------------------------------Menjalankan dan Membandingkan Hasil
# Jalankan metode Gauss-Seidel
V_final = gauss_seidel(Ybus, P_load, Q_load, V)
print("Tegangan akhir pada setiap node (Gauss-Seidel):")
print(V_final)

# Jalankan metode Jacobi
V_final_jacobi = jacobi(Ybus, P_load, Q_load, V)
print("Tegangan akhir pada setiap node (Jacobi):")
print(V_final_jacobi)


Converged in 177 iterations.
Tegangan akhir pada setiap node (Gauss-Seidel):
[1.       +0.j        2.9092329-4.4110229j]
Jacobi converged in 177 iterations.
Tegangan akhir pada setiap node (Jacobi):
[1.       +0.j        2.9092329-4.4110229j]


In [ ]:
#------------------------------------ Grafik perbandingan tegangan awal dan akhir
V_initial_magnitude = np.abs(V)
V_final_magnitude = np.abs(V_final)

plt.figure(figsize=(10, 6))
plt.plot(V_initial_magnitude, 'bo-', label='Tegangan Awal')
plt.plot(V_final_magnitude, 'ro-', label='Tegangan Akhir')
plt.xlabel('Node')
plt.ylabel('Magnitudo Tegangan (p.u.)')
plt.title('Perbandingan Tegangan Awal dan Akhir')
plt.legend()
plt.grid()
plt.savefig('voltage_comparison.png')
plt.show()

# Grafik perbandingan losses
losses_comparison = [initial_losses, total_losses]
labels = ['Losses Awal', 'Losses Akhir']

plt.figure(figsize=(10, 6))
plt.bar(labels, losses_comparison, color=['blue', 'red'])
plt.xlabel('Kondisi')
plt.ylabel('Losses (kW)')
plt.title('Perbandingan Losses')
plt.savefig('losses_comparison.png')
plt.show()

# Unduh file hasil
files.download('result.txt')
files.download('results.csv')
files.download('voltage_comparison.png')
files.download('losses_comparison.png')

NameError: name 'np' is not defined

 Program tersebut mencakup beberapa aspek kunci yang relevan untuk optimasi jaringan penyulang:

Pembacaan Data Awal: Anda telah menunjukkan bagaimana membaca data impedansi saluran (Ybus), data beban pada setiap node (P_load dan Q_load), dan tegangan awal pada setiap node (V). Data ini penting untuk memodelkan jaringan penyulang secara matematis.

Metode Gauss-Seidel: Anda mengimplementasikan metode Gauss-Seidel untuk menyelesaikan sistem persamaan non-linier yang muncul dalam analisis tegangan dalam jaringan. Metode ini digunakan untuk mencari tegangan akhir pada setiap node dalam jaringan.

Perhitungan Losses: Anda telah menambahkan fungsi untuk menghitung losses dalam jaringan berdasarkan tegangan akhir yang diperoleh dari metode Gauss-Seidel. Perhitungan ini penting untuk mengevaluasi efisiensi jaringan dan potensi penghematan yang bisa dicapai.

Evaluasi Peningkatan Pendapatan: Anda juga telah menyediakan fungsi untuk menghitung potensi peningkatan pendapatan PLN berdasarkan pengurangan losses dalam jaringan. Ini merupakan bagian penting dari analisis ekonomi yang terkait dengan implementasi teknik optimasi seperti yang diusulkan.

Dengan demikian, program tersebut secara keseluruhan memberikan kerangka kerja yang jelas dan terstruktur untuk mengeksplorasi dan mengimplementasikan optimasi jaringan penyulang menggunakan metode Gauss-Seidel. Namun, untuk keperluan tesis, pastikan untuk:

Validasi dan Verifikasi: Pastikan bahwa Anda melakukan validasi terhadap hasil-hasil yang diperoleh dengan membandingkan dengan studi kasus atau data simulasi yang relevan.

Analisis Tambahan: Selain pengurangan losses dan pendapatan, mungkin juga perlu mempertimbangkan faktor-faktor lain seperti stabilitas tegangan, kualitas daya, dan pengaruh teknis lainnya.

Penyajian dan Interpretasi: Pastikan untuk menyajikan hasil dengan jelas dan menginterpretasikan implikasi dari temuan Anda terhadap tujuan optimasi dan kinerja jaringan penyulang.

Pendahuluan:

Latar Belakang: Jelaskan pentingnya optimasi jaringan penyulang 20 kV dan masalah yang dihadapi terkait dengan losses.
Tujuan Penelitian: Nyatakan tujuan utama penelitian Anda, yaitu mengurangi losses dan meningkatkan pendapatan melalui metode Gauss-Seidel.
Ruang Lingkup Penelitian: Jelaskan lingkup penelitian Anda, misalnya wilayah UP3 Semarang.
Kajian Pustaka:

Teori Jaringan Distribusi: Jelaskan teori dasar jaringan distribusi listrik, termasuk karakteristik dan tantangan dalam pengelolaannya.
Metode Gauss-Seidel: Berikan ulasan mendalam tentang metode Gauss-Seidel, termasuk prinsip dasar dan aplikasi dalam analisis jaringan listrik.
Optimasi Losses: Tinjau literatur terkait dengan teknik-teknik pengurangan losses dalam jaringan distribusi.
Metodologi:

Model Matematika: Berikan model matematika dari jaringan penyulang 20 kV, termasuk impedansi saluran dan model beban.
Implementasi Metode Gauss-Seidel: Jelaskan bagaimana metode Gauss-Seidel diterapkan untuk mencari tegangan node dan bagaimana proses iterasi dilakukan.
Data dan Asumsi: Sebutkan data yang digunakan (misalnya data impedansi, data beban) dan asumsi yang diambil dalam analisis.
Implementasi dan Simulasi:

Persiapan Data: Tunjukkan bagaimana data diimpor dan dipersiapkan untuk analisis. Anda bisa menunjukkan contoh data yang digunakan.
Kode Program: Sertakan kode program yang telah Anda tulis, dan jelaskan setiap bagian kode tersebut. Anda bisa menggunakan program yang Anda tulis di atas.
Simulasi: Tunjukkan hasil simulasi dengan menggunakan metode Gauss-Seidel. Presentasikan hasil tegangan pada setiap node setelah iterasi.
Analisis Hasil:

Perhitungan Losses: Hitung dan presentasikan losses pada jaringan sebelum dan sesudah optimasi.
Evaluasi Ekonomi: Tunjukkan bagaimana pengurangan losses berkontribusi pada peningkatan pendapatan PLN. Buat perhitungan yang menunjukkan peningkatan pendapatan berdasarkan pengurangan losses.
Pembahasan:

Perbandingan Hasil: Bandingkan hasil yang Anda peroleh dengan studi atau metode lain jika ada. Jelaskan mengapa metode Anda efektif atau mungkin memiliki keterbatasan.
Implikasi Hasil: Diskusikan implikasi praktis dari hasil penelitian Anda bagi operasional PLN UP3 Semarang.
Kesimpulan dan Rekomendasi:

Kesimpulan: Rangkum temuan utama dari penelitian Anda. Tegaskan kembali bagaimana tujuan penelitian tercapai.
Rekomendasi: Berikan rekomendasi untuk implementasi praktis dan penelitian lebih lanjut.